In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packa ges to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.stats import zscore
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',200)

In [ ]:
df=pd.read_csv('../input/ecommerce-data/data.csv',encoding = 'ISO-8859-1')
df.head()

In [ ]:
df.shape


In [ ]:
df['InvoiceDate'].min()
df['InvoiceDate'].max()


In [ ]:
df.info()
df.isna().sum()

In [ ]:
nan_rows = df[df.isnull().T.any().T]
nan_rows.head(5)


In [ ]:
nan_rows.shape

In [ ]:
df[df['InvoiceNo']== '536544']

In [ ]:
df = df.dropna(subset=["CustomerID"])
df.shape

In [ ]:
df.isna().sum()

In [ ]:
print('Dupplicate entries: {}'.format(df.duplicated().sum()))
#df.drop_duplicates(inplace = True)

In [ ]:
df.drop_duplicates(inplace = True)
df.info

In [ ]:
df['Country'].value_counts()

In [ ]:
print(df['Country'].unique())
print('Total no. of countries from where customers belong: ',df['Country'].nunique())

In [ ]:
print('Total no. of customers: ',df['CustomerID'].nunique())
print('Total transactions done: ',df['InvoiceNo'].nunique())
print('Products sold are : ',df['StockCode'].nunique())

In [ ]:
customer_country=df[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

In [ ]:
df.describe()

In [ ]:
df.quantile([0,.01,.02,.03,.05,.1,.25,.5,.75,.9,.95,.99,1]).T
#2+% records show negative quantities

In [ ]:
df[(df['Quantity']<0)].head(5)


In [ ]:
cnt_order = df[df['Quantity']>0]['InvoiceNo'].nunique()
cnt_refund = df[df['Quantity']<0]['InvoiceNo'].nunique()
cnt_cancelled = df['InvoiceNo'].apply(lambda x:x[0]=='C').nunique()

print("Total Orders : ",cnt_order)
print("Total Refund Order : ",cnt_refund)
print("%Refund : ",cnt_refund/(cnt_order)*100,"%")
print("%Cancelled : ",cnt_cancelled/(cnt_order)*100,"%")

In [ ]:
negq = df[df['Quantity']>0]
cnt_cancelled1 = negq['InvoiceNo'].apply(lambda x:x[0]=='C').nunique()
cnt_cancelled1
negq.shape

In [ ]:
list_special_codes = negq[negq['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
list_special_codes


In [ ]:
xx = negq[negq.StockCode.isin(['POST', 'C2', 'M', 'BANK CHARGES', 'PADS', 'DOT'])]
xx.shape
xx.groupby(['StockCode','Description'])['InvoiceNo'].aggregate('count').reset_index().sort_values('StockCode', ascending=False)


In [ ]:
df_cleaned = negq[negq['StockCode']!= 'POST']
df_cleaned = df_cleaned[df_cleaned['StockCode']!= 'D']
df_cleaned = df_cleaned[df_cleaned['StockCode']!= 'C2']
df_cleaned = df_cleaned[df_cleaned['StockCode']!= 'M']
df_cleaned = df_cleaned[df_cleaned['StockCode']!= 'BANK CHARGES']
df_cleaned = df_cleaned[df_cleaned['StockCode']!= 'PADS']
df_cleaned = df_cleaned[df_cleaned['StockCode']!= 'DOT']
df_cleaned.describe()

In [ ]:
df_cleaned[(df_cleaned['UnitPrice'] == 0)].head(5)


In [ ]:
def unique_counts(data):
   for i in data.columns:
       count = data[i].nunique()
       print(i, ": ", count)
unique_counts(df_cleaned)

In [ ]:
df_cleaned['InvoiceDate'].min()
df_cleaned['InvoiceDate'].max()
df['InvoiceDate'].max()
import datetime as dt
max_date = dt.datetime(2011,12,10)
max_date

In [ ]:
df_cleaned['Total_amount'] = df_cleaned['Quantity'] * df_cleaned['UnitPrice']
df_cleaned.head()

In [ ]:
revenue_per_countries = df_cleaned.groupby(["Country"])["Total_amount"].sum().sort_values(ascending = False).reset_index()
revenue_per_countries
total = revenue_per_countries['Total_amount'].sum()
total
revenue_per_countries['Percent_Rev']= round(revenue_per_countries.Total_amount/total *100)
revenue_per_countries = revenue_per_countries.sort_values(by ='Percent_Rev')
revenue_per_countries1 = revenue_per_countries.tail(10).sort_values(by ='Percent_Rev')
revenue_per_countries1.plot('Country','Percent_Rev',kind='barh', figsize=(10,10))
plt.title("Revenue Share by Country (Top 10)")

In [ ]:
No_invoice_per_country = df_cleaned.groupby(["Country"])["InvoiceNo"].count().sort_values(ascending = False).reset_index()
total = No_invoice_per_country['InvoiceNo'].sum()
total

No_invoice_per_country['Percent_Inv']= round(No_invoice_per_country.InvoiceNo/total *100)
No_invoice_per_country = No_invoice_per_country.sort_values(by ='Percent_Inv')
No_invoice_per_country1 = No_invoice_per_country.tail(10).sort_values(by ='Percent_Inv')
No_invoice_per_country1.plot('Country','Percent_Inv',kind='barh', figsize=(10,10))
plt.title("Percent of Invoices per Country(Top 10)")

In [ ]:
tot= pd.merge(revenue_per_countries, No_invoice_per_country, on='Country', how='inner')
tot.head()
tot['Avg_Order_value']=tot.Total_amount/tot.InvoiceNo*100
tot = tot.sort_values(by ='Avg_Order_value')
tot.plot('Country','Avg_Order_value',kind='barh', figsize=(10,10))
plt.title("Average order value per Country")

In [ ]:
df_cleaned['month_year']=pd.to_datetime(df_cleaned['InvoiceDate']).dt.to_period('M')
df_cleaned.head()



In [ ]:
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

In [ ]:
L=['year','month','day','dayofweek','dayofyear','weekofyear','quarter']
df_cleaned=df_cleaned.join(pd.concat((getattr(df_cleaned['InvoiceDate'].dt,i).rename(i) for i in L),axis=1))


In [ ]:
df_cleaned['dayofweek']=df_cleaned['dayofweek']+1
df_cleaned.head()

In [ ]:
df_cleaned.dayofweek.value_counts()

In [ ]:
#highest spenders
spent_per_cust=df_cleaned.groupby(by=['CustomerID','Country'],as_index=False)['Total_amount'].sum().sort_values(by='Total_amount',ascending=False)
#spent_per_cust.columns=['CustomerID','Country','TotalSpent']
spent_per_cust.head(10)
spent_per_cust.quantile([0,.01,.02,.03,.05,.1,.25,.5,.75,.9,.95,.99,1]).T

In [ ]:
df_cleaned.insert(loc=2,column='year_month',value=df_cleaned['InvoiceDate'].map(lambda x: 100*x.year + x.month))
df_cleaned.insert(loc=5,column='hour',value=df_cleaned.InvoiceDate.dt.hour)


In [ ]:
df_cleaned.head()

In [ ]:
ax=df_cleaned.groupby('InvoiceNo')['month_year'].unique().value_counts().sort_index()
ax.plot(kind='bar', figsize=(10,10))
plt.title("Unique Invoice count by Month")

In [ ]:
ax=df_cleaned.groupby('InvoiceNo')['dayofweek'].unique().value_counts().sort_index()
ax.plot(kind='bar', x =('Mon','Tue','Wed','Thur','Fri','Sun'),figsize=(10,10))
plt.title("Unique Invoice count by Weekdays")
#ax.set_xticklabels(('Mon','Tue','Wed','Thur','Fri','Sun'),rotation='horizontal',fontsize=15);

In [ ]:
a=df_cleaned.groupby('CustomerID').sum()['Total_amount']
a= a.reset_index()
a.head()

b= df_cleaned.groupby('CustomerID')['InvoiceNo'].count()
b= b.reset_index()
b.columns = ['CustomerID', 'Frequency']
b.head()
#df_cleaned.describe()

FA= pd.merge(a, b, on='CustomerID', how='inner')
FA.head()

df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])
df_cleaned['days_diff'] = max_date - df_cleaned['InvoiceDate']
df_cleaned.head()
c = df_cleaned.groupby('CustomerID')['days_diff'].min()
c = c.reset_index()
c.head()
c['days_diff'] = c['days_diff'].dt.days
c.head()



In [ ]:
df_cust_level = pd.merge(FA, c, on='CustomerID', how='inner')
df_cust_level.columns = ['CustomerID', 'Total_amount', 'Frequency', 'Recency']
df_cust_level.head()


In [ ]:
quantiles = df_cust_level.quantile(q=[0.20,0.80])
quantiles = quantiles.to_dict()
segmented_rfm = df_cust_level

#functions to rank the 3 criteria
def RScore(x,p,d):
    if x <= d[p][0.20]:
        return 'High'
    elif x <= d[p][0.80]: 
        return 'Medium'
    else:
        return 'Low'
    
def FMScore(x,p,d):
    if x <= d[p][0.20]:
        return 'Low'
    elif x <= d[p][0.80]:
        return 'Medium'
    else:
        return 'High'

In [ ]:
segmented_rfm['r_segment'] = segmented_rfm['Recency'].apply(RScore, args=('Recency',quantiles,))
segmented_rfm['f_segment'] = segmented_rfm['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
segmented_rfm['m_segment'] = segmented_rfm['Total_amount'].apply(FMScore, args=('Total_amount',quantiles,))
segmented_rfm.head()

In [ ]:
xx=segmented_rfm.groupby(['r_segment','f_segment','m_segment']).CustomerID.count().reset_index()
xx


In [ ]:
# For clustering
segmented_rfm2=segmented_rfm[['Total_amount','Frequency','Recency']] 
segmented_rfm3 = segmented_rfm2.apply(zscore)
segmented_rfm3.head()

In [ ]:
#kmeans = KMeans(random_state=2)
#kmeans.fit(segmented_rfm3)

cluster_range = range( 1, 15 )
cluster_errors = []
for num_clusters in cluster_range:
    clusters = KMeans( num_clusters, n_init = 10 )
    clusters.fit(segmented_rfm3)
    cluster_errors.append( clusters.inertia_ )
    
    xx = clusters.fit_predict(segmented_rfm3)
    silhouette_avg = silhouette_score(segmented_rfm3,xx)
    print("The average silhouette_score for cluster ",num_clusters,": ", silhouette_avg, "with Std Scaling")
    
clusters_df = pd.DataFrame( { "num_clusters":cluster_range, "cluster_errors": cluster_errors } )
clusters_df[0:15]

In [ ]:
cluster_range = range( 1, 15 )
cluster_errors = []
for num_clusters in cluster_range:
    #clusters = KMeans(init= 'k-means++',n_clusters= num_clusters, n_init = 15,random_state=0)
    clusters = KMeans(n_clusters= num_clusters, n_init = 15,random_state=0)
    clusters.fit(segmented_rfm3)
    cluster_errors.append(clusters.inertia_)
    xx = clusters.predict(segmented_rfm3)
    #xx = clusters.fit_predict(segmented_rfm3)
    #silhouette_avg = silhouette_score(segmented_rfm3,xx)
    #print("The average silhouette_score for cluster ",num_clusters,": ", silhouette_avg, "with Std Scaling")
    
clusters_df = pd.DataFrame( { "num_clusters":cluster_range, "cluster_errors": cluster_errors } )
clusters_df[0:15]

In [ ]:
plt.figure(figsize=(12,6))
plt.plot( clusters_df.num_clusters, clusters_df.cluster_errors, marker = "o" )

In [ ]:
kmeans = KMeans(n_clusters=4, n_init = 15, random_state=0)
kmeans.fit(segmented_rfm3)
centroids = kmeans.cluster_centers_
centroid_df = pd.DataFrame(centroids, columns = list(segmented_rfm3) )
centroid_df

In [ ]:
df_labels = pd.DataFrame(kmeans.labels_ , columns = list(['labels']))
df_labels['labels'] = df_labels['labels'].astype('category')
df_labels

segmented_rfm_kmeans = segmented_rfm.join(df_labels)
segmented_rfm_kmeans.head()

In [ ]:
clus =3
segmented_rfm_kmeans[segmented_rfm_kmeans['labels']==clus].mean()
#segmented_rfm_kmeans[segmented_rfm_kmeans['labels']==clus].median()


In [ ]:
qq=segmented_rfm_kmeans[segmented_rfm_kmeans['labels']==clus].groupby(['r_segment','f_segment','m_segment']).CustomerID.count().reset_index()
qq

In [ ]:
temp_final_df = segmented_rfm_kmeans.reset_index()
cust0 = list(temp_final_df[temp_final_df['labels']==clus]['CustomerID'])

cluster0 = df_cleaned[df_cleaned['CustomerID'].isin(cust0)]
cluster0[['Quantity', 'UnitPrice','Total_amount']].mean()

In [ ]:
cluster0['Description'].value_counts()[:10]


In [ ]:
cluster0['month'].value_counts()


In [ ]:
cluster0['dayofweek'].nunique()


In [ ]:
ax=cluster0.groupby('InvoiceNo')['dayofweek'].unique().value_counts().sort_index()
ax.plot(kind='bar', x =('Mon','Tue','Wed','Thur','Fri','Sun'),figsize=(10,10))
plt.title("Unique Invoice count by Weekdays")

In [ ]:
segmented_rfm_kmeans.labels.value_counts()

In [ ]:
sns.pairplot(segmented_rfm_kmeans,diag_kind='kde',hue='labels')
#sns.pairplot(segmented_rfm_kmeans,vars=["labels","Total_amount"],diag_kind='auto',hue='labels')


In [ ]:
xx.shape

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
proj = tsne.fit_transform(segmented_rfm3)
plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=xx)
plt.title("Visualization of the clustering with TSNE", fontsize="25")

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
df_cleaned['Year']=df_cleaned.InvoiceDate.dt.year
df_cleaned['Month']=df_cleaned.InvoiceDate.dt.month
df_cleaned['Day']=df_cleaned.InvoiceDate.dt.day
df_cleaned['Date']=pd.to_datetime(df_cleaned[['Year','Month','Day']])
df_cleaned.head()


In [ ]:
df_cleaned['Week']=df_cleaned.InvoiceDate.dt.isocalendar().week
df_cleaned['Sales'] = df_cleaned.Quantity*df_cleaned.UnitPrice

In [ ]:
ds_weekly = df_cleaned.groupby(by=['Year','Week'])['Sales'].sum().reset_index()
ds_daily = df_cleaned.groupby(by=['Date'])['Sales'].sum().reset_index()
ds_daily.head()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter(x=ds_daily.Date,y=ds_daily.Sales)])
fig.update_layout(xaxis_title="Date",yaxis_title="Sales",title='Daily Trend',height=400,template='ggplot2')
fig.show()

In [ ]:
#Testing stationarity of actual series
output = adfuller(ds_daily.Sales)
print('ADF Statistic: {0:.2f} and P value:{1:.5f}'.format(*output))
#gives out high p value - meaning that the series is NOT stationary

In [ ]:
#series with 1 differencing
series_date = ds_daily.Sales.diff(1)
series_date = series_date.dropna()
output = adfuller(series_date)
print('ADF Statistic: {0:.2f} and P value:{1:.5f}'.format(*output))

#data stationary now

In [ ]:
#pcaf & acf
fig, ax = plt.subplots(1,2,figsize=(15,5))
plot_acf(series_date, ax=ax[0])
plot_pacf(series_date, ax=ax[1])
plt.show()

In [ ]:
ds_daily.shape

In [ ]:
series_date=ds_daily.Sales
split_time = 250
time_d=np.arange(len(ds_daily))
xtrain_d=series_date[:split_time]
xtest_d=series_date[split_time:]
timeTrain_d = time_d[:split_time]
timeTest_d = time_d[split_time:]
print('Full Set Size ',series_date.shape)
print('Training Set Size ',xtrain_d.shape)
print('Testing Set Size ',xtest_d.shape)

In [ ]:
timeTest_d

In [ ]:
#ARIMA 
from statsmodels.tsa.arima.model import ARIMA
s_model = ARIMA(endog=xtrain_d , order=(1, 1, 1))
s_model_fit=s_model.fit()
print(s_model_fit.summary())

In [ ]:
def plot_(t_train,t_test,x_train,x_test,x_train_pred,x_test_pred,forecast,title='Weekly'):
    xt = (max(t_test)+np.arange(len(forecast)))+1
    fig_train=go.Scatter(name='Train : Actual ',x=t_train,y=x_train,showlegend=True)
    fig_trian_pred=go.Scatter(name='Train : Predict',x=t_train,y=x_train_pred,showlegend=True)
    fig_test=go.Scatter(name='Test : Actual',x=t_test,y=x_test,showlegend=True)
    fig_test_pred=go.Scatter(name='Test : Predict',x=t_test,y=x_test_pred,showlegend=True)
    fig_forecast=go.Scatter(name='Forecast',x=xt,y=forecast,showlegend=True)

    fig = go.Figure([fig_train,fig_trian_pred,fig_test,fig_test_pred,fig_forecast])
    fig.update_layout(xaxis_title=title,yaxis_title="Sales",title=title +' Trend'
                      ,height=400,hovermode="x",template='ggplot2')
    fig.show()


In [ ]:
ytrain_pred = s_model_fit.predict()
ytest_pred = s_model_fit.predict(start=min(timeTest_d),end=max(timeTest_d),dynamic=True)

print('MSE Train :',np.sqrt(np.mean((ytrain_pred - xtrain_d)**2)))
print('MSE Test :',np.sqrt(np.mean((ytest_pred - xtest_d)**2)))
forecast = s_model_fit.forecast(20, alpha=0.05)

plot_(t_train = timeTrain_d,t_test = timeTest_d,x_train = xtrain_d,x_test = xtest_d,x_train_pred = ytrain_pred
      ,x_test_pred = ytest_pred,forecast = forecast,title='Daily')

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
s_model = SARIMAX(endog=xtrain_d , order=(1, 1, 1), seasonal_order=(1, 1, 2, 12), trend='t')
s_model_fit=s_model.fit()
print(s_model_fit.summary())

In [ ]:
ytrain_pred = s_model_fit.predict()
ytest_pred = s_model_fit.predict(start=min(timeTest_d),end=max(timeTest_d),dynamic=True)

print('MAPE Train:',np.mean(((xtrain_d - ytrain_pred)/xtrain_d)))
print('MSE Train :',np.sqrt(np.mean((ytrain_pred - xtrain_d)**2)))
print('MAPE Test:',np.mean(abs((xtest_d - ytest_pred)/xtest_d)))
print('MSE Test :',np.sqrt(np.mean((ytest_pred - xtest_d)**2)))
forecast = s_model_fit.forecast(30, alpha=0.05)
xt = max(timeTest_d)+np.arange(len(forecast))

plot_(t_train = timeTrain_d,t_test = timeTest_d,x_train = xtrain_d,x_test = xtest_d,x_train_pred = ytrain_pred
      ,x_test_pred = ytest_pred,forecast = forecast,title='Daily')